In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import cv2
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from glob import glob

from collections import OrderedDict
import torch
import gc

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

https://www.kaggle.com/c/petfinder-pawpularity-score/discussion/301686  
Petfinder コンペで優勝したGibaさんのプログラムを利用

# Install CLIP library

In [ ]:
!pip install ../input/openaiclipweights/python-ftfy-master/python-ftfy-master
!pip install ../input/openaiclipweights/clip/CLIP
!cp ../input/openaiclipweights/CLIP-main/CLIP-main/clip/bpe_simple_vocab_16e6.txt /opt/conda/lib/python3.7/site-packages/clip/.
!gzip -k /opt/conda/lib/python3.7/site-packages/clip/bpe_simple_vocab_16e6.txt
!ls /opt/conda/lib/python3.7/site-packages/clip/.

In [ ]:
import torch
import clip
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, Dataset

print("Torch version:", torch.__version__)

# Download some images from open images collection.

In [ ]:
!wget https://farm8.staticflickr.com/6036/6426668771_b5b915e46c_o.jpg
!wget https://c6.staticflickr.com/8/7457/10806045045_02d3dbdcee_o.jpg
!wget https://c1.staticflickr.com/4/3267/2888764405_0a0a608604_o.jpg
!wget https://farm8.staticflickr.com/4028/4294212194_a49663b2b9_o.jpg
!wget https://c5.staticflickr.com/9/8173/8019508216_6540c8686a_o.jpg
!wget https://farm3.staticflickr.com/1146/1357102390_943c5cb999_o.jpg

In [ ]:
files = glob('*.jpg')
print(files)

# List pretrained CLIP models available

In [ ]:
clip.available_models()

# List pretrained weights available

In [ ]:
!ls ../input/openaiclipweights/clip/CLIP/models/

# Load CLIP Vision Transformer based model

In [ ]:
model, preprocess = clip.load("../input/openaiclipweights/clip/CLIP/models/ViT-B-32.pt")
model.cuda().eval()

# For each image we will query for the following senteces and see what CLIP predicts. 
# You can add custom sentences here.

In [ ]:
QUERIES = [
    "a cat in a sofa",
    "a dog",
    "a cat",
    "a elephant",
    "a zebra",
    "a sleeping dog",
    "a sleeping cat",
    "a giraffe",
    "a poodle",
    "animal inside a car",
    "animal outside a car",
    "a sofa",
    "some animals",
    "santa claus",
    "ipod",
    "two mugs",
    "three mugs",
    "blue sky",
    "three mugs and a iphone",
] 

# Score images vs queries using clip model

In [ ]:
with torch.no_grad():
    for file in files:
        print(file)
        # Load image from file
        img = Image.open(file).convert("RGB")

        # Just show image in the notebook
        plt.imshow(cv2.resize(np.array(img), (256, 256)))
        plt.show()
        
        # Preprocess image using clip
        img = preprocess(img).unsqueeze(0).cuda()
        
        # Get Image embeddings
        image_embeddings = model.encode_image(img)
        image_embeddings /= image_embeddings.norm(dim=-1, keepdim=True)
        
        
        score = []
        for query in QUERIES:
            texts = clip.tokenize(query).cuda()
            
            # Get Text Embeddings
            text_embeddings = model.encode_text(texts)
            text_embeddings /= text_embeddings.norm(dim=-1, keepdim=True)
            
            # Calc dot product between image and text embeddings
            sc = float((image_embeddings @ text_embeddings.T).cpu().numpy())
            score.append(sc)
        
        print( pd.DataFrame({'query': QUERIES, 'score': score}).sort_values('score', ascending=False) )
        print('')
        print('-------------------------')
        print('')


画像読み込み関数

In [ ]:
## 追加

import io
import requests

In [ ]:
def softmax(x):
    u = np.sum(np.exp(x))
    return np.exp(x)/u

In [ ]:
## URL　画像読み込み実験
img_urls =[
#    "https://drive.google.com/file/d/1UxtPN-SnAIO9hLOZ303eTMTrWb6Ssq55/view"]
#           "https://www.yodosha.co.jp/rnote/gazou_qa/images/9784758115537_2pic1_l.png"
#           "https://phio.panasonic.co.jp/kinen/cxr/cr_image/case82.jpg"] # 胸部Xp
#    "https://www.harasanshin.or.jp/img/seikei-daitaikotsu_kossetsu1.jpg"] # 大腿骨転子部骨折
    "https://univ-journal.jp/wp-content/uploads/2019/08/pixta_46842118_S-1200x675.jpg"] # 岡山大学
    
img = Image.open((io.BytesIO(requests.get(img_urls[0]).content))).convert("RGB")

plt.imshow(cv2.resize(np.array(img), (256, 256)))
plt.show()

外部画像

In [ ]:
# 問い合わせ文章　長い文章でも良い
QUERIES = [
    "a Ryobi systems",
    "japanese university",
    "a chest x-ray",
    "a hip x-ray",
    "an anterior view of hip x-ray",
    "a lateral view of hip x-ray",    
    "a normal hip x-ray",
    "a fractured hip x-ray",
    "database",
    "a cup",
    "a trophy",
    "a elephant",
    "a zebra",
    "a sleeping dog",
    "a sleeping cat",
]

In [ ]:
with torch.no_grad():
    for img_url in img_urls:
        print(file)
        # Load image from file
        #img = Image.open(file).convert("RGB")
        img = Image.open((io.BytesIO(requests.get(img_url).content))).convert("RGB")

        # Just show image in the notebook
        plt.imshow(cv2.resize(np.array(img), (256, 256)))
        plt.show()
        
        # Preprocess image using clip
        img = preprocess(img).unsqueeze(0).cuda()
        
        # Get Image embeddings
        image_embeddings = model.encode_image(img)
        image_embeddings /= image_embeddings.norm(dim=-1, keepdim=True)
        
        
        score = []
        for query in QUERIES:
            texts = clip.tokenize(query).cuda()
            
            # Get Text Embeddings
            text_embeddings = model.encode_text(texts)
            text_embeddings /= text_embeddings.norm(dim=-1, keepdim=True)
            
            # Calc dot product between image and text embeddings
            sc = float((image_embeddings @ text_embeddings.T).cpu().numpy())
            score.append(sc)
        
        scoreN = [n*100 for n in score]
        scoreSoftmax = softmax(scoreN)
        scoreSoftmax = np.round(scoreSoftmax *100)
        scoreNP = scoreSoftmax

        print( pd.DataFrame({'query': QUERIES, 'score': score, '%': scoreNP}).sort_values('score', ascending=False) )
        #print( pd.DataFrame({'query': QUERIES, 'score': score}).sort_values('score', ascending=False) )
        print('')
        print('-------------------------')
        print('')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# GPUからCPUに移してtensor -> numpy　
device_cpu = torch.device('cpu')
image_embeddings_cpu = image_embeddings.to(device_cpu)
image_embeddings_numpy = image_embeddings_cpu.clone().numpy()

# (1,512) なので1次元配列に
image_embeddings_numpy_1d = image_embeddings_numpy.flatten()

In [ ]:
plt.plot(image_embeddings_numpy_1d)

In [ ]:
def convertTorch2Numpy(embeddings):
    # GPUからCPUに移してtensor -> numpy　
    device_cpu = torch.device('cpu')
    embeddings_cpu = embeddings.to(device_cpu)
    embeddings_numpy = embeddings_cpu.clone().detach().numpy()
    # (1,512) なので1次元配列に
    embeddings_numpy_1d = embeddings_numpy.flatten()
    return embeddings_numpy_1d

In [ ]:
def getTextEmbeddings(text):
    # single text
    texts = clip.tokenize(text).cuda()
    # Get Text Embeddings
    text_embeddings = model.encode_text(texts)
    text_embeddings /= text_embeddings.norm(dim=-1, keepdim=True)
    return text_embeddings

In [ ]:
def calcDotProduct(image_embeddings, text_embeddings):
    # Calc dot product between image and text embeddings
    sc = float((image_embeddings @ text_embeddings.T).cpu().detach().numpy())
    return sc 

In [ ]:
# 
image_embeddings_np = convertTorch2Numpy(image_embeddings)

In [ ]:
# 異なる単語で問い合わせた場合、どれだけ値が異なるかを可視化
# single text embeddings
text1 = "a Ryobi systems"
text2 = "a elephant",

text_embeddings1 = getTextEmbeddings(text1)
text_embeddings_np1 = convertTorch2Numpy(text_embeddings1)
text_embeddings2 = getTextEmbeddings(text2)
text_embeddings_np2 = convertTorch2Numpy(text_embeddings2)

# graph1
fig, ax = plt.subplots()
x = np.linspace(0,511,512)
ax.plot(x,image_embeddings_np)
ax.plot(x,text_embeddings_np1, color='green')
ax.set_title(calcDotProduct(image_embeddings, text_embeddings1))

# graph2
fig, ax = plt.subplots()
x = np.linspace(0,511,512)
ax.plot(x,image_embeddings_np)
ax.plot(x,text_embeddings_np2, color='red')
ax.set_title(calcDotProduct(image_embeddings, text_embeddings2))


In [ ]:
# graph1-2
fig, ax = plt.subplots()
x = np.linspace(0,511,512)
ax.plot(x,text_embeddings_np1, color='green')
ax.plot(x,text_embeddings_np2, color='red')

In [ ]:
# 散布図
plt.scatter(text_embeddings_np1,text_embeddings_np2, c=x, cmap='Blues')
plt.colorbar()

In [ ]:
softmax(score)

In [ ]:
softmaxScore = softmax2(score)

In [ ]:
score

In [ ]:
sum(softmaxScore)

In [ ]:
softmaxScore